In [14]:
import pandas as pd
import utils 
import plotly.graph_objects as go 
import instrument

In [15]:
pair = "CAD_CHF"
granularity = "H1"
ma_list=[16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [16]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ["time", "volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [17]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [18]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-09-30T04:00:00.000000000Z,0.71188,0.71200,0.71134,0.71179,0.713366,0.717651
1,2022-09-30T05:00:00.000000000Z,0.71180,0.71234,0.71146,0.71190,0.713118,0.717499
2,2022-09-30T06:00:00.000000000Z,0.71193,0.71354,0.71155,0.71337,0.713002,0.717380
3,2022-09-30T07:00:00.000000000Z,0.71335,0.71502,0.71216,0.71428,0.712947,0.717284
4,2022-09-30T08:00:00.000000000Z,0.71424,0.71516,0.71392,0.71445,0.712894,0.717202


In [19]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 
    return 0

In [20]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [21]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
25,2022-10-03T05:00:00.000000000Z,0.71641,0.71670,0.71576,0.71598,0.715024,0.715018,0.000005,-0.000172,1
83,2022-10-05T15:00:00.000000000Z,0.72038,0.72110,0.71984,0.72016,0.723569,0.723768,-0.000199,0.000158,-1
131,2022-10-07T15:00:00.000000000Z,0.72494,0.72583,0.72402,0.72411,0.721911,0.721906,0.000005,-0.000231,1
176,2022-10-11T12:00:00.000000000Z,0.72240,0.72391,0.72158,0.72384,0.723714,0.723847,-0.000134,0.000026,-1
227,2022-10-13T15:00:00.000000000Z,0.72356,0.72684,0.72328,0.72660,0.722817,0.722647,0.000169,-0.000106,1


In [22]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [23]:
df_trades["GAIN"].sum()

16.30000000000134

In [24]:
df_plot = df_ma.iloc[0:100].copy()

In [25]:
df_plot.shape

(100, 10)

In [26]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()